## 크롤링을 완료한 후에 csv 파일 이름을 꼭 변경 후 다시 코드 재시작하기!!!!!!!!!!!!!!
## 안그러면 매번 같은 이름으로 저장돼서 나중에 자르기 불편함

In [7]:
########## 매인코드 
#### 식당 페이지 하단 1,2,3,4,5 넘어가도록 설정
#### 리뷰 페이지 더보기 버튼 4번 클릭 (총 50개의 리뷰 크롤링)
#### 검색은 (중랑구 nn동 '음식카테고리') 로 검색 (ex. 중랑구 묵동 일식 / 중랑구 면목동 한식) 


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import (
    NoSuchElementException,
    StaleElementReferenceException,
    ElementClickInterceptedException,
    TimeoutException
)
from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import datetime
import os


########################################
# 기본 설정
########################################
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")

# ✅ 홈으로 진입
url = "https://map.naver.com/p"

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)
driver.get(url)

########################################
# 공통 유틸
########################################
def to_default():
    driver.switch_to.default_content()

def switch_to_search_iframe(timeout=20):
    to_default()
    WebDriverWait(driver, timeout).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe"))
    )

def switch_to_entry_iframe(timeout=20):
    to_default()
    WebDriverWait(driver, timeout).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe"))
    )

def js_click(el):
    driver.execute_script("arguments[0].click();", el)

def safe_click(el):
    try:
        el.click()
    except ElementClickInterceptedException:
        js_click(el)

def scroll_center(el):
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    time.sleep(0.2)

def safe_text(css, default=""):
    try:
        els = driver.find_elements(By.CSS_SELECTOR, css)
        if els:
            return (els[0].text or "").strip()
    except Exception:
        pass
    return default


########################################
# ✅✅✅ 중간 저장(append) + 재시작 스킵용 함수들
# (중요) 반드시 "루프에서 호출하기 전" 위에 정의되어 있어야 함
########################################
def append_place_to_csv(place_dict, out_path):
    """
    place_dict 1개(가게 1개)를 CSV에 누적 저장(append)
    - 리뷰는 1리뷰=1행
    - 리뷰 없으면 1행 남김
    """
    place_name = place_dict.get("name", "")
    category = place_dict.get("category", "")
    addr1 = place_dict.get("address1", "")
    addr2 = place_dict.get("address2", "")
    address = (addr1 + " " + addr2).strip()
    reviews = place_dict.get("reviews", []) or []

    rows = []
    if not reviews:
        rows.append({
            "name": place_name,
            "category": category,
            "address": address,
            "review_index": None,
            "review_text": "",
            "review_info": "",
            "visit_time": ""
        })
    else:
        for idx, r in enumerate(reviews, start=1):
            txt = (r.get("review_text", "") or "").replace("\n", " ").strip()
            info = (r.get("review_info", "") or "").replace("\n", " ").strip()
            vt = (r.get("visit_time", "") or "").replace("\n", " ").strip()
    
            rows.append({
                "name": place_name,
                "category": category,
                "address": address,
                "review_index": idx,
                "review_text": txt,
                "review_info": info,      
                "visit_time": vt          
            })


    df = pd.DataFrame(rows)
    file_exists = os.path.exists(out_path)
    df.to_csv(out_path, mode="a", header=not file_exists, index=False, encoding="utf-8-sig")


def load_visited_from_csv(out_path):
    """
    기존 CSV가 있으면 name+address로 visited set을 만들어 반환
    """
    if not os.path.exists(out_path):
        return set()

    try:
        df = pd.read_csv(out_path, encoding="utf-8-sig")
        visited = set()

        if "name" not in df.columns or "address" not in df.columns:
            return set()

        for _, row in df[["name", "address"]].dropna().iterrows():
            n = str(row["name"]).strip()
            a = str(row["address"]).strip()
            if n or a:
                visited.add((n, a))

        print(f"✅ 기존 CSV에서 visited 로드: {len(visited)}개")
        return visited

    except Exception as e:
        print("⚠️ 기존 CSV 로드 실패:", repr(e))
        return set()


########################################
# ✅ (추가) 홈에서 검색창 클릭 후 키워드 검색
########################################
def search_from_home(keyword: str, timeout=20):
    """
    1) 홈에서 검색창(#home_search_input_box) 클릭
    2) input에 keyword 입력 후 Enter
    3) searchIframe 로딩까지 대기
    """
    keyword = (keyword or "").strip()
    if not keyword:
        raise ValueError("검색어가 비어있음")

    to_default()

    # 홈 검색창 클릭
    search_btn = WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#home_search_input_box > div > div"))
    )
    safe_click(search_btn)
    time.sleep(0.3)

    # 검색 input 잡기 (홈에서 보통 input 하나가 잡힘)
    search_input = WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input.input_search"))
    )

    # 입력 + Enter
    search_input.clear()
    search_input.send_keys(keyword)
    search_input.send_keys(Keys.ENTER)

    # 검색 결과 iframe 로딩 대기
    switch_to_search_iframe(timeout=timeout)

    # 검색 결과 리스트가 뜰 때까지(안 뜨는 케이스 대비)
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div#_pcmap_list_scroll_container"))
    )


########################################
# 1번 코드: 현재 페이지 끝까지 스크롤
########################################
def scroll_list_to_end(pause=1.0, max_try=120):
    scroll_box = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div#_pcmap_list_scroll_container"))
    )

    last_height = 0
    same_count = 0

    for _ in range(max_try):
        height = driver.execute_script("return arguments[0].scrollHeight", scroll_box)
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_box)
        time.sleep(pause)

        if height == last_height:
            same_count += 1
        else:
            same_count = 0

        if same_count >= 3:
            break

        last_height = height


########################################
# ✅ 리뷰 탭 클릭: nth-child 금지, '리뷰' 텍스트로 클릭
########################################
def click_review_tab_by_text(timeout=10):
    try:
        tab_area = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.place_fixed_maintab"))
        )
        tabs = tab_area.find_elements(By.CSS_SELECTOR, "a")

        for a in tabs:
            txt = (a.text or "").strip().replace("\n", " ")
            if "리뷰" in txt:
                scroll_center(a)
                safe_click(a)
                time.sleep(1.2)
                return True

        return False
    except TimeoutException:
        return False

def wait_review_list_loaded(timeout=12):
    end = time.time() + timeout
    while time.time() < end:
        try:
            if driver.find_elements(By.CSS_SELECTOR, "#_review_list > li"):
                return True
        except Exception:
            pass
        time.sleep(0.3)
    return False


########################################
# 2번 코드: 리뷰 '리스트 더보기' 버튼 4회 클릭 (원본 셀렉터 유지)
########################################
MORE_BTN_SELECTOR_SPAN = (
    "#app-root > div > div > div:nth-child(7) > div:nth-child(3) "
    "> div.place_section.k1QQ5 > div.NSTUp > div > a > span"
)

def click_review_list_more_fixed(times=4):
    for i in range(times):
        switch_to_entry_iframe()
        try:
            span = WebDriverWait(driver, 8).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, MORE_BTN_SELECTOR_SPAN))
            )
            a = span.find_element(By.XPATH, "./parent::a")

            before_cnt = len(driver.find_elements(By.CSS_SELECTOR, "#_review_list > li"))

            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", a)
            time.sleep(0.25)
            safe_click(a)
            time.sleep(1.3)

            after_cnt = len(driver.find_elements(By.CSS_SELECTOR, "#_review_list > li"))
            print(f"🔁 리뷰 리스트 더보기 {i+1}회: {before_cnt} -> {after_cnt}")

            if after_cnt <= before_cnt:
                print("⛔ 리뷰 수 증가 없음 → 더보기 종료")
                break

        except TimeoutException:
            print("⛔ 리뷰 리스트 더보기 버튼 없음 → 종료")
            break
        except StaleElementReferenceException:
            time.sleep(0.3)
            continue


########################################
# 2번 코드: 리뷰 수집 (원본 유지)
########################################
REVIEW_LI = "#_review_list > li"
REVIEW_WRAP = "div.pui__vn15t2"
READ_MORE_CANDIDATES = ["a.pui__jhpEy", "a[role='button']", "a"]
REVIEW_INFO_SEL = "div.pui__QztK4Q > div.Vk05k > div"
VISIT_TIME_SEL  = "div.pui__-0Ter1 > a"

def scroll_under_header(el):
    driver.execute_script("arguments[0].scrollIntoView({block:'start'});", el)
    time.sleep(0.15)
    header_h = driver.execute_script("""
        const h = document.querySelector('#_header');
        return h ? h.getBoundingClientRect().height : 0;
    """)
    driver.execute_script("window.scrollBy(0, arguments[0] + 12);", header_h)
    time.sleep(0.15)

def click_read_more_if_exists(wrap):
    for sel in READ_MORE_CANDIDATES:
        try:
            anchors = wrap.find_elements(By.CSS_SELECTOR, sel)
        except Exception:
            anchors = []

        for a in anchors:
            try:
                txt = (a.text or "").strip()
                if "더보기" in txt:
                    scroll_under_header(a)
                    try:
                        a.click()
                    except ElementClickInterceptedException:
                        js_click(a)
                    time.sleep(0.2)
                    return True
            except StaleElementReferenceException:
                continue
    return False

def extract_review_text_from_wrap(wrap):
    raw = (wrap.text or "").strip()
    return raw.replace("더보기", "").strip()

    
def crawl_visible_reviews(wait_sec=10):
    switch_to_entry_iframe()
    wait_local = WebDriverWait(driver, wait_sec)
    wait_local.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, REVIEW_LI)))

    reviews = []
    li_elems = driver.find_elements(By.CSS_SELECTOR, REVIEW_LI)

    for idx in range(len(li_elems)):
        for _ in range(3):
            try:
                switch_to_entry_iframe()
                li = driver.find_elements(By.CSS_SELECTOR, REVIEW_LI)[idx]

                # ✅ (기존) 리뷰 본문
                wrap = li.find_element(By.CSS_SELECTOR, REVIEW_WRAP)
                click_read_more_if_exists(wrap)
                text = extract_review_text_from_wrap(wrap)

                # ✅ (추가) 리뷰 정보
                review_info = ""
                try:
                    info_el = li.find_element(By.CSS_SELECTOR, REVIEW_INFO_SEL)
                    review_info = (info_el.text or "").strip()
                except Exception:
                    review_info = ""

                # ✅ (추가) 방문 시간대
                visit_time = ""
                try:
                    vt_el = li.find_element(By.CSS_SELECTOR, VISIT_TIME_SEL)
                    visit_time = (vt_el.text or "").strip()
                except Exception:
                    visit_time = ""

                # ✅ 저장 형태: dict
                reviews.append({
                    "review_text": (text or "").strip(),
                    "review_info": review_info,
                    "visit_time": visit_time
                })
                break

            except (StaleElementReferenceException, NoSuchElementException, IndexError):
                time.sleep(0.2)
                continue

    return reviews



################################################################################
# ✅ 주소 펼치기 버튼: O8qbU 블록 내부에서 span._UCia 찾아 부모 a 클릭
################################################################################
def expand_address_if_possible(timeout=8):
    try:
        switch_to_entry_iframe()
        block = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.O8qbU.tQY7D"))
        )
        spans = block.find_elements(By.CSS_SELECTOR, "span._UCia")
        if spans:
            a = spans[0].find_element(By.XPATH, "./ancestor::a[1]")
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", a)
            time.sleep(0.2)
            safe_click(a)
            time.sleep(1.0)
            return True
    except Exception:
        pass
    return False


########################################
# entryIframe에서 "정보 + 리뷰" 뽑는 함수
########################################
def scrape_current_place():
    switch_to_entry_iframe()
    time.sleep(1.0)

    name = safe_text("#_title div span.GHAhO", "")
    category = safe_text("#_title > div > span.lnJFt", "")

    # ✅ 네가 준 주소 셀렉터 (그대로)
    ADD1_CSS = "#app-root > div > div > div:nth-child(6) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.tQY7D > div > div.Y31Sf > div:nth-child(1)"
    ADD2_CSS = "#app-root > div > div > div:nth-child(6) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.tQY7D > div > div.Y31Sf > div:nth-child(2)"

    expand_address_if_possible()
    add1 = safe_text(ADD1_CSS, "")
    add2 = safe_text(ADD2_CSS, "")
    print(f"🏠 [ADDR CHECK] {name} | add1='{add1}' | add2='{add2}'")
    if not add1 and not add2:
        print("⚠️ [ADDR EMPTY] 주소가 비었습니다 (셀렉터/펼치기 실패 가능)")

    reviews = []
    ok = click_review_tab_by_text()
    if ok:
        loaded = wait_review_list_loaded(timeout=12)
        if loaded:
            click_review_list_more_fixed(times=4)
            try:
                reviews = crawl_visible_reviews()
            except Exception:
                reviews = []
        else:
            reviews = []
    else:
        reviews = []

    return {"name": name, "category": category, "address1": add1, "address2": add2, "reviews": reviews}


########################################
# 검색 결과 한 페이지에서 모든 카드(현재 로딩된 것) 순회
########################################
def get_visible_card_count():
    switch_to_search_iframe()
    return len(driver.find_elements(By.CSS_SELECTOR, "li.UEzoS"))

def click_card_by_index(idx):
    """
    searchIframe에서 idx번째 카드의 '가게명(span.TYaxT)'만 클릭
    → 사진/광고 클릭 문제 방지
    """
    switch_to_search_iframe()
    nth = idx + 1

    title_css = (
        f"#_pcmap_list_scroll_container > ul > li:nth-child({nth}) "
        "> div.CHC5F > div.bSoi3 > a > span.TYaxT"
    )

    try:
        title = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, title_css))
        )
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", title)
        time.sleep(0.2)
        safe_click(title)
        time.sleep(1.5)
        return True
    except TimeoutException:
        print(f"⚠️ 카드 {nth} 클릭 실패 (가게명 못 찾음)")
        return False


########################################
# ✅ 실행
########################################
keyword = input("네이버 지도 검색어 입력: ").strip()
search_from_home(keyword)

# ✅ 고정 파일명 (재시작 누적/스킵하려면 고정이어야 함)
out_path = "naver_map_reviews_.csv"
print("📌 중간 저장 CSV:", out_path)

# ✅ 재시작 스킵용 visited 미리 로드
visited = load_visited_from_csv(out_path)

results = []

page_round = 1
switch_to_search_iframe()

print(f"\n===== PAGE ROUND {page_round} =====")

scroll_list_to_end()
time.sleep(0.6)

card_count = get_visible_card_count()
print(f"📦 현재 페이지 카드 수: {card_count}")

for i in range(card_count):
    try:
        ok = click_card_by_index(i)
        if not ok:
            continue

        data = scrape_current_place()

        # ✅ 재시작/중복 스킵 키: (name, address)
        address = (data.get("address1", "") + " " + data.get("address2", "")).strip()
        key = (data.get("name", "").strip(), address)

        if key in visited:
            print(f"↩️ 이미 저장됨 스킵: {key[0]} / {key[1]}")
            switch_to_search_iframe()
            continue

        visited.add(key)
        results.append(data)

        # ✅ 가게 1개 끝날 때마다 CSV 누적 저장
        append_place_to_csv(data, out_path)
        print(f"✅ 수집: {data.get('name','')} / 리뷰 {len(data.get('reviews',[]))}개")
        print("💾 누적 저장 완료:", out_path)

        switch_to_search_iframe()

    except Exception as e:
        print("⚠️ 카드 처리 중 에러:", repr(e))
        switch_to_search_iframe()
        continue

print("\n========== 최종 결과 ==========")
print("총 가게 수:", len(results))
if results:
    print("첫 가게 예시:", results[0]["name"], "/", results[0]["category"], "/", results[0]["address1"], "/", results[0]["address2"])
    print("첫 가게 리뷰 샘플:", results[0]["reviews"][:3])

driver.quit()


네이버 지도 검색어 입력:  중랑구 묵동 고기


📌 중간 저장 CSV: naver_map_reviews_.csv

===== PAGE ROUND 1 =====
📦 현재 페이지 카드 수: 50
🏠 [ADDR CHECK] 임금돼지 태릉 | add1='도로명서울 중랑구 동일로 932 2단지상가 103호복사' | add2='지번서울 중랑구 묵동 170-1복사'
🔁 리뷰 리스트 더보기 1회: 10 -> 20
🔁 리뷰 리스트 더보기 2회: 20 -> 30
🔁 리뷰 리스트 더보기 3회: 30 -> 40
🔁 리뷰 리스트 더보기 4회: 40 -> 50
✅ 수집: 임금돼지 태릉 / 리뷰 50개
💾 누적 저장 완료: naver_map_reviews_.csv
🏠 [ADDR CHECK] 우소담 본점 | add1='도로명서울 중랑구 동일로 932 자이아파트 2단지 104호복사' | add2='지번서울 중랑구 묵동 170-1복사'
⛔ 리뷰 리스트 더보기 버튼 없음 → 종료
✅ 수집: 우소담 본점 / 리뷰 10개
💾 누적 저장 완료: naver_map_reviews_.csv
🏠 [ADDR CHECK] 쪽갈비대통령 먹골본점 | add1='도로명서울 중랑구 동일로163길 46복사' | add2='지번서울 중랑구 묵동 237-138복사'
⛔ 리뷰 리스트 더보기 버튼 없음 → 종료
✅ 수집: 쪽갈비대통령 먹골본점 / 리뷰 10개
💾 누적 저장 완료: naver_map_reviews_.csv
🏠 [ADDR CHECK] CAMPxGRILL | add1='도로명서울 중랑구 동일로 929 2층복사' | add2='지번서울 중랑구 묵동 238-90복사'
🔁 리뷰 리스트 더보기 1회: 10 -> 20
🔁 리뷰 리스트 더보기 2회: 20 -> 30
🔁 리뷰 리스트 더보기 3회: 30 -> 40
🔁 리뷰 리스트 더보기 4회: 40 -> 50
✅ 수집: CAMPxGRILL / 리뷰 50개
💾 누적 저장 완료: naver_map_reviews_.csv
🏠 [ADDR CHECK] 백운축산 정육식당 먹골점 | add1='도로명서울 중랑구 공릉로 32 공감대아파트 